<a href="https://colab.research.google.com/github/AYA0HASSAN/Pose_detection/blob/main/NotebookV1(Without_Normalization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install mediapipe opencv-python

In [ ]:
# import mediapipe and opencv
import mediapipe as mp
import cv2
import glob
import numpy as np
import csv
import os
from mat4py import loadmat
import scipy.io as sio
import math

In [ ]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_drawing_styles = mp.solutions.drawing_styles


In [ ]:
paths,paths_mat=[],[]
# get the path/directory
folder_dir ="C:\\Users\\ahmad\Downloads\AFLW2000"

# iterate over files in
# that directory
for images in glob.iglob(f'{folder_dir}/*'):

    # check if the image ends with png
    if (images.endswith(".jpg")):
#         print(images)
        paths.append(images)

In [ ]:
len(paths)

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 

    for idx, file in enumerate(paths):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        # Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#         results = holistic.process(image)
#         print(results.face_landmarks)
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        
        break


In [ ]:
results.face_landmarks.landmark[0]

In [ ]:
num_coords = len(results.face_landmarks.landmark)
num_coords

In [ ]:
landmarks = []
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val)]
# landmarks
angles=['pitch','yaw','roll']
data = landmarks+angles

In [ ]:
with open('landmarks_data.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(data)

In [ ]:
paths[0].split('\\')[-1].replace('jpg','')

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for idx, file in enumerate(paths):
        count=0
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
#         Convert the BGR image to RGB before processing.
        results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        try:
           
            face = results.face_landmarks.landmark
            NoseX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width
            NoseY=results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y   *image_height
            LfeoX = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].x *image_width
            LfeoY = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE_OUTER].y* image_height
            
            dist = math.dist([NoseX, NoseY], [LfeoX, LfeoY])
        
            
            
            
            
            
            face_row = list(np.array([[   ((landmark.x *image_width)-NoseX)/dist, 
                                            ((landmark.y*image_height)-NoseY)/dist] for landmark in face]).flatten())
            
            random_file = file.split('\\')[-1].replace('.jpg' , '')
            mat_file = sio.loadmat('C:\\Users\\ahmad\\Downloads\\AFLW2000\\'+ random_file+ '.mat')
            pose_para = mat_file["Pose_Para"][0][:3]
            pitch = pose_para[0]
            yaw = pose_para[1]
            roll = pose_para[2]
            face_row.insert(((468*2)+0),pitch)
            face_row.insert(((468*2)+1),yaw)
            face_row.insert(((468*2)+2),roll)
            
            with open('landmarks_data.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(face_row) 
                
        except:
            pass
        

# Read and Prepare Data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('landmarks_data.csv')
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y465,x466,y466,x467,y467,x468,y468,pitch,yaw,roll
0,-0.159295,0.300003,-0.169120,0.067671,-0.146352,0.109180,-0.190952,-0.301066,-0.161892,-0.022124,...,-0.640003,0.132219,-0.604989,0.751723,-0.761274,0.807323,-0.806983,-0.399231,0.018227,0.085676
1,-0.334478,0.261430,-0.472663,-0.051103,-0.284650,0.053159,-0.358159,-0.332285,-0.480441,-0.137888,...,-0.525273,0.037087,-0.505745,0.562256,-0.565438,0.632099,-0.617696,0.470065,1.189533,0.300959
2,-0.981481,1.076963,-0.956155,0.954189,-0.976846,0.985318,-1.013231,0.778098,-0.954629,0.903266,...,0.634062,-0.919076,0.647000,-0.675460,0.588512,-0.651586,0.567591,-0.184650,0.881137,-0.236852
3,0.127447,0.433119,-0.086203,0.200610,0.035008,0.228611,-0.239562,-0.172509,-0.133581,0.104239,...,-0.642603,0.021166,-0.595805,0.629668,-0.972822,0.673793,-1.040024,-0.175379,0.299208,-0.373374
4,-0.078761,0.400796,-0.344684,0.090758,-0.104594,0.147864,-0.317465,-0.235742,-0.383646,-0.000681,...,-0.595326,-0.004369,-0.552671,0.427108,-0.790101,0.482372,-0.853066,-0.882169,1.198003,-1.033374


In [ ]:
#Features
x=df.drop(['pitch','yaw','roll'], axis=1)
#Pitch label
yp=df['pitch']
#Yaw label
yy=df['yaw']
#roll label
yr=df['roll']
#labels
yall =df[['pitch','yaw','roll']]

In [ ]:
#************************************************ PITCH ****************************************************************

X_train_p, X_val_p, y_train_p, y_val_p = train_test_split(x, yp, test_size=0.2, random_state=1234)

X_val_p, X_test_p, y_val_p, y_test_p = train_test_split(X_val_p, y_val_p, test_size=0.5, random_state=1234)


#************************************************* YAW ******************************************************************

X_train_y, X_val_y, y_train_y, y_val_y = train_test_split(x, yy, test_size=0.2, shuffle=True,random_state=1234)

X_val_y, X_test_y, y_val_y, y_test_y = train_test_split(X_val_y, y_val_y, test_size=0.5, shuffle=True,random_state=1234)


#*********************************************** Roll *******************************************************************

X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(x, yr, test_size=0.2,shuffle=True, random_state=1234)

X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_val_r, y_val_r, test_size=0.5,shuffle=True, random_state=1234)


#*************************************************** ALL ****************************************************************


X_train_all, X_val_all, y_train_all, y_val_all = train_test_split(x, yall, test_size=0.2,shuffle=True, random_state=1234)

X_val_all, X_test_all, y_val_all, y_test_all = train_test_split(X_val_all, y_val_all, test_size=0.5,shuffle=True, random_state=1234)



# Train Machine Learning Models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge , ridge_regression,Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor,BaggingRegressor,ExtraTreesRegressor
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
import sklearn.linear_model

# YAW

In [ ]:
pipelines = {
#     'lr_pca':make_pipeline(StandardScaler(),PCA(n_components=0.99), LinearRegression()),
    'rf_pca':make_pipeline(StandardScaler(),PCA(n_components=0.99), RandomForestRegressor())
#     ,'lr_wpca':make_pipeline(PCA(n_components=0.99),LinearRegression()),
#     'rf_wpca':make_pipeline(PCA(n_components=0.99),RandomForestRegressor()),
#     'DT'    :make_pipeline(DecisionTreeRegressor()),
#     'DTpca' :make_pipeline(PCA(n_components=0.99),DecisionTreeRegressor()),
#     'rfpcap' :make_pipeline(   RandomForestRegressor(n_estimators=30, n_jobs=-1))
# 
}



In [ ]:
yaw_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_y, y_train_y)
    yaw_fit_models[algo] = model


In [ ]:
Train_pred_yaw      = yaw_fit_models['rf_pca'].predict(X_train_y)
Validation_pred_yaw = yaw_fit_models['rf_pca'].predict(X_val_y)
yaw_hat=yaw_fit_models['rf_pca'].predict(X_test_y)

In [ ]:
print("Train MSE "       , mean_squared_error( y_train_y,Train_pred_yaw ))
print("Validation MSE "  , mean_squared_error( y_val_y,Validation_pred_yaw ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_y,Train_pred_yaw))
print("Validation r2_score "       , r2_score( y_val_y, Validation_pred_yaw))
print("Test r2_score "             , r2_score(y_test_y , yaw_hat))
print("\n###########################\n")

print("Train score "                   , yaw_fit_models['rf_pca'].score(X_train_y , y_train_y))
print("Validation Score "              , yaw_fit_models['rf_pca'].score(X_val_y, y_val_y))
print("test score: "                   ,yaw_fit_models ['rf_pca'].score(X_test_y, y_test_y))
 
print("\n###########################\n")

Train MSE  0.01096723181404754
Validation MSE  0.02235874416339032

###########################

Train r2_score  0.9671595765146147
Validation r2_score  0.935790893015142
Test r2_score  0.939158917015116

###########################

Train score  0.9671595765146147
Validation Score  0.935790893015142
test score:  0.939158917015116

###########################



# Roll

In [ ]:
pipelines = {
#     'rf_pca':make_pipeline(StandardScaler(),PCA(n_components=0.99), RandomForestRegressor()),
#     'DTpca' :make_pipeline(PCA(n_components=0.99),DecisionTreeRegressor()),
    'rfpcap' :make_pipeline(PCA(n_components=0.99), RandomForestRegressor(n_estimators=200))


}


In [ ]:
roll_fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train_r, y_train_r)
    roll_fit_models[algo] = model


In [ ]:
Train_pred_roll     = roll_fit_models  ['rfpcap'].predict(X_train_r)
Validation_pred_roll = roll_fit_models ['rfpcap'].predict(X_val_r)
roll_hat             =roll_fit_models  ['rfpcap'].predict(X_test_r)

In [ ]:
print("Train MSE "       , mean_squared_error( y_train_r,Train_pred_roll ))
print("Validation MSE "  , mean_squared_error( y_val_r,Validation_pred_roll ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_r,Train_pred_roll))
print("Validation r2_score "       , r2_score( y_val_r, Validation_pred_roll))
print("Test r2_score "             , r2_score(y_test_r , roll_hat))
print("\n###########################\n")

print("Train score "                   , roll_fit_models['rfpcap'].score(X_train_r , y_train_r))
print("Validation Score "              , roll_fit_models['rfpcap'].score(X_val_r, y_val_r))
print("test score: "                   , roll_fit_models ['rfpcap'].score(X_test_r, y_test_r))
 
print("\n###########################\n")


Train MSE  0.057589714398586164
Validation MSE  0.022289056932878814

###########################

Train r2_score  0.8858221349732149
Validation r2_score  0.7053034867828494
Test r2_score  0.6886814209568144

###########################

Train score  0.8858221349732149
Validation Score  0.7053034867828494
test score:  0.6886814209568144

###########################



# ROLL WITH VOTING

In [ ]:
r1 = svm.SVR(kernel='rbf')
r2 = DecisionTreeRegressor()
r3 = RandomForestRegressor(n_estimators=200)
r4=VotingRegressor([('r1',r1),('r2' , r2),('r3',r3)])

In [ ]:
# X_train_r=PCA().fit_transform(X_train_r)


In [ ]:
r4.fit(X_train_r, y_train_r)

VotingRegressor(estimators=[('r1', SVR()), ('r2', DecisionTreeRegressor()),
                            ('r3', RandomForestRegressor(n_estimators=200))])

In [ ]:
Train_pred_roll     = r4.predict(X_train_r)
Validation_pred_roll = r4.predict(X_val_r)
roll_hat             =r4.predict(X_test_r)

In [ ]:
print("Train MSE "       , mean_squared_error( y_train_r,Train_pred_roll ))
print("Validation MSE "  , mean_squared_error( y_val_r,Validation_pred_roll ))

print("\n###########################\n")

print("Train r2_score "            , r2_score(y_train_r,Train_pred_roll))
print("Validation r2_score "       , r2_score( y_val_r, Validation_pred_roll))
print("Test r2_score "             , r2_score(y_test_r , roll_hat))
print("\n###########################\n")

print("Train score "                   , r4.score(X_train_r , y_train_r))
print("Validation Score "              , r4.score(X_val_r, y_val_r))
print("test score: "                   , r4.score(X_test_r, y_test_r))
 
print("\n###########################\n")


Train MSE  0.0858647063087905
Validation MSE  0.012797042895045725

###########################

Train r2_score  0.8297638918707624
Validation r2_score  0.8308028943522826
Test r2_score  0.5563622026403514

###########################

Train score  0.8297638918707624
Validation Score  0.8308028943522826
test score:  0.5563622026403514

###########################



In [ ]:
# ######## DTpca
# Train MSE  0.0
# Validation MSE  0.04477750019570877

# ###########################

# Train r2_score  1.0
# Validation r2_score  0.4079707715766818
# Test r2_score  0.6242046134613088

# ###########################

# Train score  1.0
# Validation Score  0.4079707715766818
# test score:  0.6242046134613088

# ###########################


#**********************************************************
 ######### rfpcap

# Train MSE  0.08568100768216784
# Validation MSE  0.015235995963917124

# ###########################

# Train r2_score  0.8301280943540559
# Validation r2_score  0.7985560851911272
# Test r2_score  0.6849495987237703

# ###########################

# Train score  0.8301280943540559
# Validation Score  0.7985560851911272
# test score:  0.6849495987237703

# ###########################


# ########### rfpcap with standard

# Train MSE  0.07622373914182135
# Validation MSE  0.01627450535743072

# ###########################

# Train r2_score  0.8488781566212209
# Validation r2_score  0.7848253518481536
# Test r2_score  0.6701656068233688

# ###########################

# Train score  0.8488781566212209
# Validation Score  0.7848253518481536
# test score:  0.6701656068233688

# ###########################




    
    




In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:


fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model



In [ ]:


fit_models['rc'].predict(X_test)



In [ ]:


for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))



In [ ]:
import pickle 

In [ ]:


with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)



In [ ]:


with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)



In [ ]:
# cap = cv2.VideoCapture(0)
# # Initiate holistic model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         # Recolor Feed
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False        
        
#         # Make Detections
#         results = holistic.process(image)
#         # print(results.face_landmarks)
        
#         # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
#         # Recolor image back to BGR for rendering
#         image.flags.writeable = True   
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         # 1. Draw face landmarks
#         mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
#                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
#                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                                  )
        
        
                        
#         cv2.imshow('Raw Webcam Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()